### Environment

In [ ]:
%cd external/YOLOX/
%pip install -r requirements.txt && python setup.py develop

%cd ../external/deep-person-reid/
%pip install -r requirements.txt && python setup.py develop

%cd ../external/fast_reid/
%pip install -r docs/requirements.txt


In [11]:
%mv /home/Deep-OC-SORT/external/weights/* /home/share/model/DeepOC-SORT/

In [1]:
!python setup.py develop

running develop
/opt/conda/lib/python3.7/site-packages/setuptools/command/easy_install.py:159: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running egg_info
creating capstone.egg-info
writing capstone.egg-info/PKG-INFO
writing dependency_links to capstone.egg-info/dependency_links.txt
writing top-level names to capstone.egg-info/top_level.txt
writing manifest file 'capstone.egg-info/SOURCES.txt'
reading manifest file 'capstone.egg-info/SOURCES.txt'
writing manifest file 'capstone.egg-info/SOURCES.txt'
running build_ext
Creating /opt/conda/lib/python3.7/site-packages/capstone.egg-link (link to .)
Adding capstone 0.0.0 to easy-install.pth file

Installed /hom

### Data Preparation

In [ ]:
!unzip /home/share/data_warehouse/MOTChallenge/MOT17-FRCNN.zip -d /home/share/datasets/mot
!unzip /home/share/data_warehouse/MOTChallenge/MOT20.zip -d /home/share/datasets

In [ ]:
# 转为COCO格式
# replace "dance" with mot17/mot20 for others
!python data/tools/convert_dance_to_coco.py 

### Evaluate

In [ ]:
%exp=baseline
# Flags to disable all the new changes
!python main.py --exp_name $exp --post --emb_off --cmc_off --aw_off --new_kf_off --grid_off --dataset mot17
!python main.py --exp_name $exp --post --emb_off --cmc_off --aw_off --new_kf_off --grid_off -dataset mot20 --track_thresh 0.4
!python main.py --exp_name $exp --post --emb_off --cmc_off --aw_off --new_kf_off --grid_off --dataset dance --aspect_ratio_thresh 1000

In [ ]:
%exp=best_paper_ablation
!python main.py --exp_name $exp --post --grid_off --new_kf_off --dataset mot17 --w_assoc_emb 0.75 --aw_param 0.5
!python main.py --exp_name $exp --post --grid_off --new_kf_off --dataset mot20 --track_thresh 0.4 --w_assoc_emb 0.75 --aw_param 0.5
!python main.py --exp_name $exp --post --grid_off --new_kf_off --dataset dance --aspect_ratio_thresh 1000 --w_assoc_emb 1.25 --aw_param 1

In [ ]:
# for ArTIST
%exp=artist_mot17_eval
!python main.py --exp_name $exp --emb_off --cmc_off --aw_off --grid_off --new_kf_off --dataset mot17_mini

In [4]:
!python external/TrackEval/scripts/run_mot_challenge.py \
  --SPLIT_TO_EVAL val \
  --METRICS HOTA CLEAR Identity \
  --TRACKERS_TO_EVAL best_paper_ablations \
  --GT_FOLDER results/gt/ \
  --TRACKERS_FOLDER results/trackers/ \
  --BENCHMARK DANCE


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/Deep-OC-SORT/external/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : results/gt/                   
TRACKERS_FOLDER      : results/trackers/             
OUTPUT_FOLDER      

### Test

In [ ]:
!python main.py --exp_name DeepOC_mot17-test --dataset mot17 --test_dataset

--

In [18]:
import json

file_path = "/home/share/datasets/mot/annotations/val_half.json"
with open(file_path, 'r') as fp:
    img_info = json.load(fp)
    seqs = [i['file_name'] for i in img_info['videos']]
    img_path = {}
    images = img_info['images']
    img0 = images[0]
    img_seq = img0['file_name'].split('/')[0]
    print(img_seq)
    print(img_info['images'][0]['width'])

MOT17-02-FRCNN
1920


In [11]:
import torch
a = torch.tensor([[[1,2]],
                  [[3,4]],
                  [[5,6]]])
print(a.shape)
b = a.permute(1,0,2)
c = b.permute(1,0,2)
print(b.shape)
print(c.shape)

torch.Size([3, 1, 2])
torch.Size([1, 3, 2])
torch.Size([3, 1, 2])


In [17]:
import numpy as np

a = np.array([[1,2,3,4],
              [5,6,7,8]])
b = torch.from_numpy(a)

print(torch.sum(b>50))

tensor(0)
